# Algorithm 3.1

In [6]:
function sgn(x)::Float64
    if x < 0
        return -1.0
    elseif x==0
        return 0.0
    else
        return 1.0
    end
end

function search(x₀::Real, f::Function, f_diff::Function; h₀=1.0)::Float64
    # initialize
    x=x₀
    h=h₀
    h=sgn(f_diff(x))*abs(h)
    X1=x
    X2=x+h
    
    if f(X1)<f(X2)
        while f(X1)<f(X2)
            h=2h
            X1=X2
            X2=X1+h

            x=X1
            h=h/2.0
        end
    else
        while f(X1)>f(X2)
            h=h/2.0
            X2=X1-h

            x=X2
            h=2h
        end
    end
    
    return x
end

search (generic function with 2 methods)

In [8]:
using PyPlot
using LinearAlgebra

In [24]:
f(x)=-(x-4)^4+3
f_diff(x)=-4*(x-4)^3
x₀=10.0
h₀=1e-8

1.0e-8

In [25]:
x=search(x₀, f, f_diff, h₀=h₀)

4.001766746454088